PyGSLIB
========

Cova3 test
---------------

This is a simple example on how to use raw cova3 to fit variograms



In [5]:
#general imports
import matplotlib.pyplot as plt   
import pygslib as gslib                  
import numpy as np 
import pandas as pd


#make the plots inline
%matplotlib inline  

## The gslib paramater file

This is the gslib parameter file:
```
                  Parameters for VMODEL
                  *********************

START OF PARAMETERS:
vmodel.var                   \file for variogram output
1   10                       \number of directions and lags
 0.0   0.0    1.0            \azm, dip, lag distance
2    0.2                     \nst, nugget effect
1    0.4  0.0   0.0   0.0    \it,cc,ang1,ang2,ang3
         10.0   5.0  10.0    \a_hmax, a_hmin, a_vert
1    0.4  0.0   0.0   0.0    \it,cc,ang1,ang2,ang3
         10.0   5.0  10.0    \a_hmax, a_hmin, a_vert
```


Fit the variogram
-----

We need first the rotation matrix


In [6]:
#rotatios matrix (one per structure)
rmatrix_d1=gslib.setrot(ang1=0,ang2=0,ang3=0,anis1=0.5,anis2=1,ind=1,maxrot=2)  #rotation structure 1
rmatrix_d2=gslib.setrot(ang1=0,ang2=0,ang3=0,anis1=0.5,anis2=1,ind=2,maxrot=2)  #rotation structure 2

rmatrix=rmatrix_d1+rmatrix_d2

print rmatrix

[[[  1.21679645e-08   1.00000000e+00   0.00000000e+00]
  [ -2.00000000e+00   2.43359290e-08   0.00000000e+00]
  [  0.00000000e+00   0.00000000e+00   1.00000000e+00]]

 [[  1.21679645e-08   1.00000000e+00   0.00000000e+00]
  [ -2.00000000e+00   2.43359290e-08   0.00000000e+00]
  [  0.00000000e+00   0.00000000e+00   1.00000000e+00]]]


In [7]:
parameters_mod = { 
                'x1'     :  0,            # X coordinates, point 1
                'y1'     :  0,            # Y coordinates, point 1
                'z1'     :  0,            # Z coordinates, point 1
                'x2'     :  0,            # X coordinates, point 2
                'y2'     :  0,            # Y coordinates, point 2
                'z2'     :  0,            # Z coordinates, point 2
                'nst'    :  2,            # number of nested structures, array('i') with bounds (ivarg), 
                                          # ivarg is variogram number
                'c0'     :  [0.2],        # nugget,  array('f') with bounds (ivarg)  
                'it'     :  [1, 1],        # structure type,  array('i') with bounds (ivarg)              
                'cc'     :  [0.4, 0.4],   # variance, array('f') with bounds (nvarg*nst[0])
                'aa'     :  [10., 10.],      # parameter a (or range), array('f') with bounds (nvarg*nst[0])
                'irot'   :  1,            # index of the rotation matrix for the first nested structure
                                          # the second nested structure will use irot+1, the third irot+2, and so on
                'rotmat' :  rmatrix}       # rotation matrices (output of the funtion setrot)

In [17]:
# this is the covariance between the points x1, x2

h=np.array([0, 0.0001, 1,2,3,4,5,6,7,8,9,10])
vario=np.zeros(12)

for i in range(len(h)):
    parameters_mod['y2']=h[i]
    cmax,cova=gslib.cova3(parameters_mod)
    vario[i]=cmax-cova
    


results = pd.DataFrame ({'h': h, 'vario': vario}) 

print results

          h     vario
0    0.0000  0.000000
1    0.0001  0.200012
2    1.0000  0.319600
3    2.0000  0.436800
4    3.0000  0.549200
5    4.0000  0.654400
6    5.0000  0.750000
7    6.0000  0.833600
8    7.0000  0.902800
9    8.0000  0.955200
10   9.0000  0.988400
11  10.0000  1.000000


## The expacted results are:
    
```
 ID      Lag        vario
   1     0.000      0.00000     
   1     0.000      0.20001     
   2     1.000      0.31960     
   3     2.000      0.43680     
   4     3.000      0.54920     
   5     4.000      0.65440     
   6     5.000      0.75000     
   7     6.000      0.83360     
   8     7.000      0.90280     
   9     8.000      0.95520     
  10     9.000      0.98840     
  11    10.000      1.00000     
```

In [21]:
# testiong different direction
h=np.array([0, 0.0001, 1,2,3,4,5,6,7,8,9,10])
parameters_mod['y2']=0
vario=np.zeros(12)

for i in range(len(h)):
    parameters_mod['x2']=h[i]
    cmax,cova=gslib.cova3(parameters_mod)
    vario[i]=cmax-cova
    


results = pd.DataFrame ({'h': h, 'vario': vario}) 

print results

          h     vario
0    0.0000  0.000000
1    0.0001  0.200024
2    1.0000  0.436800
3    2.0000  0.654400
4    3.0000  0.833600
5    4.0000  0.955200
6    5.0000  1.000000
7    6.0000  1.000000
8    7.0000  1.000000
9    8.0000  1.000000
10   9.0000  1.000000
11  10.0000  1.000000


```
   ID    Lag        Vario
   1     0.000      0.00000
   1     0.000      0.20002
   2     1.000      0.43680
   3     2.000      0.65440
   4     3.000      0.83360
   5     4.000      0.95520
   6     5.000      1.00000
   7     6.000      1.00000
   8     7.000      1.00000
   9     8.000      1.00000
  10     9.000      1.00000
  11    10.000      1.00000

```